In [1]:
from surface_distance import compute_surface_distances, compute_robust_hausdorff, compute_dice_coefficient
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os

In [2]:
def compute_metrics(mask_gt, mask_pred):
    distances = compute_surface_distances(mask_gt, mask_pred, [1,1,1])
    hausdorff = compute_robust_hausdorff(distances, 95)
    dice = compute_dice_coefficient(mask_gt, mask_pred)
    return dice, hausdorff

In [3]:
columns = ["ID", "2D-Dice", "2D-Hausdorff", "3D-Dice", "3D-Hausdorff", "ENS-Dice", "ENS-Hausdorff"]
metrics = pd.DataFrame(columns=columns)

In [ ]:
dir_path_gt = "segmentations/GT-segmentation/"
dir_path_2D = "segmentations/2D-segmentation/"
dir_path_3D = "segmentations/3D-segmentation/"
dir_path_ENS = "segmentations/ENS-segmentation/"

for image in os.listdir(dir_path_gt):
    subject_id = image[0:20]
    print("Working on: " + subject_id)
    
    mask_gt = sitk.GetArrayFromImage(sitk.ReadImage(dir_path_gt + image))
    mask_2D = sitk.GetArrayFromImage(sitk.ReadImage(dir_path_2D + image))
    mask_3D = sitk.GetArrayFromImage(sitk.ReadImage(dir_path_3D + image))
    mask_ENS = sitk.GetArrayFromImage(sitk.ReadImage(dir_path_ENS + image))
    
    mask_gt = np.array(mask_gt, dtype=bool)
    mask_2D = np.array(mask_2D, dtype=bool)
    mask_3D = np.array(mask_3D, dtype=bool)
    mask_ENS = np.array(mask_ENS, dtype=bool)


    metrics_2D = compute_metrics(mask_gt, mask_2D)
    metrics_3D = compute_metrics(mask_gt, mask_3D)
    metrics_ENS = compute_metrics(mask_gt, mask_gt)

    metrics = metrics.append({"ID": subject_id, "2D-Dice": metrics_2D[0], "2D-Hausdorff": metrics_2D[1], "3D-Dice": metrics_3D[0], "3D-Hausdorff": metrics_3D[1],
                              "ENS-Dice": metrics_ENS[0], "ENS-Hausdorff": metrics_ENS[1]}, ignore_index=True)

In [ ]:
metrics

In [8]:
metrics = metrics.sort_values(["ID"], ascending=True)

In [9]:
metrics = metrics.reset_index(drop=True)

In [10]:
name_mapping = pd.read_csv("mapping/name_mapping.csv")

In [11]:
name_mapping["ENS-Dice"] = metrics["ENS-Dice"]
name_mapping["ENS-Hausdorff"] = metrics["ENS-Hausdorff"]

In [12]:
name_mapping.to_csv("mapping/name_mapping.csv", index=False)